In [7]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt

plt.style.use('ggplot')
% matplotlib inline

# Data Loading

There are two .csv files to load:

**1) 4sqForMSBA.csv** containing the restaurant ratings (in addition to many other restaurant ratings)

**2) tips.csv** containing the comments

Since the 4sqForMSBA.csv file contains more restaurant IDs than tips.csv, we will need to remove the superfluous restaurant IDs from 4sqForMSBA.csv

In [32]:
# Similar to R's setwd()
path = 'C:/Users/JeffM/Documents/Grad School/Predictive Analytics/Final Project/Data/'

# Contains restaurant IDs and ratings (the target variable)
ratings = pd.read_csv(path + '4sqForMSBA.csv'
                 , names = ['id', 'DateValue', 'url', 'city', 'country', 'postalCode', 'state', 'price'
                            ,'rating', 'ratingSignals', 'photos', 'likes', 'name'
                            , 'dropthis']  # This column contains unintelligible data, will be dropped later
                 , encoding = "ISO-8859-1"  # Required to avoid encoding error
                 , skiprows = 1).drop('dropthis', axis = 1)  # Dropping the extra column


# Contains the comments for the restaurants
df = pd.read_csv(path + 'tips.csv'
                   , encoding = "ISO-8859-1")  # Required to avoid encoding error


# Limiting ratings to the restaurants in tips.csv
restaurantList = pd.DataFrame(df['venue_id'].unique(), columns = ['id'])  # Generating a list of the restaurant IDs

ratings = ratings.merge(restaurantList, left_on = 'id', right_on = 'id')

ratings.ix[:1]

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,DateValue,url,city,country,postalCode,state,price,rating,ratingSignals,photos,likes,name
0,44e5e786f964a52073371fe3,2015-06-28,http://grubwithus.com,New York,United States,10003,NY,2,NaN,NaN,294.0,163.0,Cafetasia
1,44e5e786f964a52073371fe3,2015-03-14,http://grubwithus.com,New York,United States,10003,NY,2,6.9,337.0,295.0,162.0,Cafetasia
2,44e5e786f964a52073371fe3,2015-03-28,http://grubwithus.com,New York,United States,10003,NY,2,7.0,286.0,294.0,162.0,Cafetasia
3,44e5e786f964a52073371fe3,2015-07-12,http://grubwithus.com,New York,United States,10003,NY,2,NaN,NaN,294.0,163.0,Cafetasia
4,44e5e786f964a52073371fe3,2015-03-31,http://grubwithus.com,New York,United States,10003,NY,2,7.0,286.0,294.0,162.0,Cafetasia


In [88]:
df.ix[:1]

,venue_id,user_id,id,likes,createdAt,collected,tip,tokenized_comments,prepared_comments
0,457c232bf964a520fe3e1fe3,32,481f806f70c603bbbf0c8eb4,17,1210024047,2015-01-12 11:27:14.180,Go to Barbone (13th and B?) and order up off t...,"[Go, to, Barbone, (, 13th, and, B, ?, ), and, ...","[go, to, barbon, (, 13th, and, b, ?, ), and, o..."
1,3fd66200f964a52000e71ee3,43,48edee3170c603bbca0c8eb4,20,1223552561,2015-01-14 10:51:02.087,Fat Cat75 Christopher @ 7th Ave.Pool Ping Pong...,"[Fat, Cat75, Christopher, @, 7th, Ave.Pool, Pi...","[fat, cat75, christoph, @, 7th, ave.pool, ping..."


# Data Processing

In [66]:
# Converting to date time columns
ratings['DateValue'] = pd.to_datetime(ratings['DateValue'])
df['collected'] = pd.to_datetime(df['collected'])


# Year
df['Year'] = df['collected'].map(lambda date: date.year)
ratings['Year'] = ratings['DateValue'].map(lambda date: date.year)

# Month
df['Month'] = df['collected'].map(lambda date: date.month)
ratings['Month'] = ratings['DateValue'].map(lambda date: date.month)

# Day Of Week
df['DayOfWeek'] = df['collected'].map(lambda date: date.dayofweek)
ratings['DayOfWeek'] = ratings['DateValue'].map(lambda date: date.dayofweek)

# Hour
# ratings only contains the date, whereas df (comments) has the time stamp
df['collected'] = df['collected'].map(lambda date: date.hour)

#### Tokenizing comments

In [51]:
# Creating an additional column for the tokenized version of the comments
df['tokenized_comments'] = df.apply(lambda row: nltk.word_tokenize(row['tip']), axis=1)

#### Converting comments to lower case and stemming them

In [52]:
stemmer = nltk.stem.PorterStemmer()

df['prepared_comments'] = df.apply(lambda row: [stemmer.stem(str.lower(word)) for word in row['tokenized_comments']]
                                   , axis = 1)

In [50]:
# nltk.sentim_analyzer.unigram_word_feats(df['tokenized_comments'], min_freq = 4)

In [86]:
import pandas_profiling

pandas_profiling.ProfileReport(ratings[['DateValue', 'city', 'photos', 'price' ,'rating']])

C:\Anaconda3\lib\site-packages\pandas\core\generic.py:3430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


Number of variables,5
Number of observations,860810
Total Missing (%),1.2%
Total size in memory,39.4 MiB
Average record size in memory,48.0 B
Numeric,2
Categorical,2
Date,1
Text (Unique),0
Rejected,0
Distinct count,236


In [90]:
pandas_profiling.ProfileReport(df[['likes', 'collected']])

C:\Anaconda3\lib\site-packages\pandas\core\generic.py:3430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


Number of variables,2
Number of observations,116819
Total Missing (%),0.0%
Total size in memory,1.8 MiB
Average record size in memory,16.0 B
Numeric,1
Categorical,0
Date,1
Text (Unique),0
Rejected,0
Distinct count,25157
